# NB for testing whole pipeline

In [155]:
%load_ext autoreload
%autoreload 2

import sys
import re
import functools

import dspy
from loguru import logger
from dspy.primitives.assertions import assert_transform_module, backtrack_handler

from ai_assistant.main import store_webpage
from ai_assistant.utils.db_utils import collection_cache
from ai_assistant.generator import init_gemini_pro
from ai_assistant.generator.qa import wiki_assistant
from ai_assistant.retriever.vdb_retrieve import Retrieval

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [156]:
logger.remove()
logger.add(sys.stderr, level="DEBUG")

3

## Define some constants to facilitate testing

In [33]:
URL = "https://en.wikipedia.org/wiki/Bhagavad_Gita"
URL_LOCAL = URL.split("/")[-1]
URL_LOCAL = re.sub(r"\W+", "", URL_LOCAL)
CHROMA_COLLECTION_NAME = f"wiki_{URL_LOCAL}"
CHROMADB_DIR = "../db/"
UPDATE = False
TEMPERATURE = 0.1

## Index URL if not already indexed
Check if data from this wiki page already indexed or if update is enabled
If not following steps:
- Crawl data from wiki page
- Index to vector db collection
- Cache indexing infor to redis

In [3]:

conn = collection_cache()
logger.info("Check if data from this wiki page already indexed")
if not conn.check(URL) or UPDATE:
    logger.info(f"(re)indexing data from {URL}")
    store_webpage(URL, CHROMA_COLLECTION_NAME, CHROMADB_DIR)

2024-05-13 14:17:52.325 | INFO     | __main__:<module>:2 - Check if data from this wiki page already indexed


## Ask Question(s)

In [136]:
# Init Generator and Retriever
retriever = Retrieval(CHROMA_COLLECTION_NAME, CHROMADB_DIR)
gemini = init_gemini_pro(temperature=TEMPERATURE)
dspy.settings.configure(lm=gemini, temperature=TEMPERATURE, max_tokens=1024, rm=retriever)

In [172]:

dspy.configure(trace=[])
assistant = assert_transform_module(
    wiki_assistant(), backtrack_handler
    # functools.partial(backtrack_handler, max_backtracks=2)
)
# assistant = wiki_assistant()
logger.info(f"Ask questions based on the wiki page about {URL_LOCAL}.")

2024-05-14 14:00:38.511 | INFO     | __main__:<module>:7 - Ask questions based on the wiki page about Bhagavad_Gita.


In [179]:
QUESTION = "What is described in the Samkhya Yoga chapter?"

In [180]:
response = assistant(QUESTION)
print(f"Response: \n {response.answer}")
print()

2024-05-14 14:02:21.778 | DEBUG    | ai_assistant.generator.qa:forward:36 - Before assertion = Prediction(
    rationale='**Answer:**\n\nThe Samkhya Yoga chapter, also known as the "Book of Doctrines" or "Self-Realization," initiates the philosophical discussions and teachings found in the Bhagavad Gita. It addresses Arjuna\'s doubts and introspection about the meaning and purpose of life as he faces a war he questions. Krishna, as his guide, imparts knowledge about the eternal nature of the soul (atman) and the transient nature of the body. The chapter explores concepts such as rebirth (samsara), the universal self present in everyone, various yoga practices, and the divinity within. It encapsulates the framework of the Samkhya and Yoga schools of Hindu philosophy and serves as an overview for the remaining chapters of the Bhagavad Gita.',
    answer='The Samkhya Yoga chapter, also known as the "Book of Doctrines" or "Self-Realization," initiates the philosophical discussions and teac

Response: 
 The Samkhya Yoga chapter, also known as the "Book of Doctrines" or "Self-Realization," initiates the philosophical discussions and teachings found in the Bhagavad Gita. It addresses Arjuna's doubts and introspection about the meaning and purpose of life as he faces a war he questions. Krishna, as his guide, imparts knowledge about the eternal nature of the soul (atman) and the temporary nature of the body. The chapter summarizes the Hindu idea of rebirth, samsara, the eternal self in each person (self), the universal self present in everyone, various types of yoga, divinity within, the nature of knowledge of the self, and other concepts. It provides a framework for the remaining sixteen chapters of the Bhagavad Gita and reflects the ideas and concepts of the Samkhya and Yoga schools of Hindu philosophy.



In [163]:
# samp1 = "Context: ABCD"
# samp2 = "**Context:**"
# reg = re.compile(r"^Context:|^.*.*Context")
# bool(re.match(reg, samp1))

True

In [169]:
gemini.inspect_history(5)




Answer the question within 200 words based on the given context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Past Answer: past output with errors

Instructions: Some instructions you must satisfy

Answer: concise, coherent and complete answer, based on context

---

Context:
[1] «the bhagavad gita is referenced in the brahma sutras and numerous scholars wrote commentaries on it, including shankara, bhaskara, abhinavagupta, ramanuja, and madhvacharya. [ 310 ] [ 311 ] many of these commentators state that the gita is " meant to be a moksa - shastra ( moksasatra ), and not a dharmasastra, an arthasastra or a kamasastra ". [ 312 ]»
[2] «bhagavad gita integrates various schools of thought, notably vedanta, samkhya and yoga, and other theistic ideas. it remains a popular text for commentators belonging to various philosophical schools. however, its composite nature also leads to varying interpretations of the text and historic scholars h

'\n\n\nAnswer the question within 200 words based on the given context.\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nQuestion: ${question}\n\nPast Answer: past output with errors\n\nInstructions: Some instructions you must satisfy\n\nAnswer: concise, coherent and complete answer, based on context\n\n---\n\nContext:\n[1] «the bhagavad gita is referenced in the brahma sutras and numerous scholars wrote commentaries on it, including shankara, bhaskara, abhinavagupta, ramanuja, and madhvacharya. [ 310 ] [ 311 ] many of these commentators state that the gita is " meant to be a moksa - shastra ( moksasatra ), and not a dharmasastra, an arthasastra or a kamasastra ". [ 312 ]»\n[2] «bhagavad gita integrates various schools of thought, notably vedanta, samkhya and yoga, and other theistic ideas. it remains a popular text for commentators belonging to various philosophical schools. however, its composite nature also leads to varying interpretations of the text